# Fine-tune a YOLO12 detection model

This notebook will train a [YOLO12](https://github.com/ultralytics/ultralytics) model for tank detection using publicly available annotated images of tanks. As the notebook will run the training with `PyTorch`, it is recommended to have GPUs available.
**This notebook expects that the dataset created in the notebook [01_Prepare](./01_Prepare.ipynb) has been saved to disk and `dataset_dir` points to it.**

In [ ]:
from orion.config.settings import settings

dataset_dir = settings.ORION_HOME_DIR / "dataset"

### Train a YOLO12 model

We'll train a `YOLO12` base model on the dataset we created in the previous notebook. We use the `yolo12n.pt` (nano) base model, which provides a good comprise between size and speed, but other models are available from ultralytics. Simply replace `yolo12n.pt` with the size you want (`n`, `s`, `m`, `l` or `x`).

In [ ]:
from pathlib import Path

from orion.yolo.yolo import train

model_dir = Path() / "runs" / "train"
results_train = train(
    "yolo12n.pt",
    data=dataset_dir / "dataset.yaml",
    epochs=60,
    batch=64,
    output=model_dir,
    exist_ok=True,
)

### Use the fine-tuned model to predict detections

Once our model is trained, we can use it to predict detections on our test dataset. Simply pass the path to the best model weights and the directory containing the test images to the `predict` function.

In [ ]:
from orion.yolo.yolo import predict

best_model = model_dir / "weights" / "best.pt"
results_predict = predict(best_model, dataset_dir / "images" / "test")

# Load the path of the prediction model results
results_predict_dir = Path(results_predict[0].save_dir) # type: ignore

### Evaluate model predictions

We can then evaluate our model by loading the predictions in a Fiftyone dataset view of our test data. We first load the dataset and then select the images tagged as part of the test set.

In [ ]:
import fiftyone as fo
from fiftyone.types.dataset_types import YOLOv5Dataset

# The splits to load
splits = ["train", "val", "test"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset()
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir, dataset_type=YOLOv5Dataset, split=split, tags=split
    )

In [ ]:
# The test split of the dataset
test_view = dataset.match_tags("test")

Then we can load the model predictions as fields of the images in our test view.

In [ ]:
from orion.yolo.utils import add_yolo_detections

prediction_field = "yolo12"
predictions_dir = results_predict_dir / "labels"
add_yolo_detections(
    test_view,
    prediction_field=prediction_field,
    predictions_dir=predictions_dir,
    class_list=["AFV", "APC", "MEV", "LAV"],
)

Finally, we can evaluate our model's predictions and print the mean Average Precision (mAP).

In [ ]:
detection_results = test_view.evaluate_detections(
    prediction_field,
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

In [ ]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

In [ ]:
detection_results.print_report()